In [1]:
Nbr_Files_to_process   =  60
schema                 = 'aemo' 
ws                     = 'sqlengines'  
lh                     = 'power'
temporary_storage      = '/tmp'
engine                 = 'pyarrow'

In [ ]:
table_base_url         =  f'abfss://{ws}@onelake.dfs.fabric.microsoft.com/{lh}.Lakehouse/Tables/{schema}/'
onelake_file_root_path =  f"{lh}.Lakehouse/Files/{schema}"

In [ ]:
!pip  install -q duckdb --upgrade
#!pip install azure-identity
#!pip install azure-storage-file-datalake

**_<u><mark>Import Python Libraries</mark></u>_**

In [3]:
from   deltalake.writer import try_get_deltatable , write_deltalake
from   deltalake        import DeltaTable
import os
import glob
from   azure.core.credentials     import AccessToken
from   azure.storage.filedatalake import DataLakeServiceClient
from   azure.identity             import DefaultAzureCredential
import duckdb
from   datetime import datetime
import time
from   psutil import *
import re
import requests
from   shutil import unpack_archive
from   urllib.request import urlopen
from   concurrent.futures import ThreadPoolExecutor, as_completed

**_<u><mark>Authentication</mark></u>_**

In [4]:
try:
       os.environ['azure_storage_token'] = notebookutils.credentials.getToken('storage')
       print(' all good you are in fabric notebook')
except:
      from   azure.identity import InteractiveBrowserCredential
      os.environ['azure_storage_token'.upper()] = InteractiveBrowserCredential().get_token("https://storage.azure.com/.default").token
duckdb.sql(f""" CREATE or replace SECRET onelake ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{os.getenv('azure_storage_token')}')   """)

 all good you are in fabric notebook


┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [5]:
vCPU = str(cpu_count()) + " vCPU"
mem=round(virtual_memory().total/(1024 * 1024 * 1024),0)
runtime = vCPU+' '+str(mem)+'GB'
runtime

'2 vCPU 15.0GB'

# Ingest Data

In [6]:
def download_web(url,Path,total_files):
    if not os.path.exists(temporary_storage+Path):
      os.makedirs(temporary_storage+Path, exist_ok=True)
    

    result = urlopen(url).read().decode('utf-8')
    pattern = re.compile(r'[\w.]*.zip')
    filelist1 = pattern.findall(result)
    filelist_unique = dict.fromkeys(filelist1)
    filelist =sorted(filelist_unique, reverse=True)

    onelake_folder = onelake_file_root_path + Path
    create_directory_if_not_exist(fs,onelake_folder)
    paths = fs.get_paths(path=onelake_folder)
    current = [os.path.basename(path.name) for path in paths ]

    #current =  [os.path.basename(x) for x in glob.glob(Path+'*.zip')]


    files_to_upload = list(set(filelist) - set(current))
    files_to_upload = list(dict.fromkeys(files_to_upload))[:total_files]
    print(str(len(files_to_upload)) + ' New File Downloaded')
    if len(files_to_upload) != 0 :
      for x in files_to_upload:
           with requests.get(url+x, stream=True) as resp:
            if resp.ok:
              with open(f"temporary_storage{Path}{x}", "wb") as f:
               for chunk in resp.iter_content(chunk_size=4096):
                f.write(chunk)
    return "done"

In [7]:
def uncompress(zip_file, source_path, destination_path):
    """Uncompress a zip file to the destination path."""
    try:
        unpack_archive(os.path.join(source_path, zip_file), destination_path, 'zip')
        print(f"Uncompressed: {zip_file}")
    except Exception as e:
        print(f"Failed to uncompress {zip_file}: {e}")

def unzip(source, destination):
    """Unzip all zip files from the source directory to the destination directory."""
    if not os.path.exists(destination):
        os.makedirs(destination, exist_ok=True)

    # Get list of zip files in the source directory
    zip_files = [os.path.basename(x) for x in glob.glob(os.path.join(source, '*.zip'))]

    # Get list of already uncompressed files in the destination directory
    uncompressed_files = [
        os.path.basename(x).replace('.CSV', '.zip').replace('.json', '.zip')
                          .replace('SCADA', 'FacilityScada').replace('-', '')
        for x in glob.glob(os.path.join(destination, '*.[Cc][Ss][Vv]')) + 
                  glob.glob(os.path.join(destination, '*.json'))
    ]

    # Determine files to uncompress (delta)
    files_to_uncompress = list(set(zip_files) - set(uncompressed_files))
    files_to_uncompress = list(dict.fromkeys(files_to_uncompress))  # Remove duplicates

    print(f"{len(files_to_uncompress)} new file(s) to uncompress.")

    if files_to_uncompress:
        with ThreadPoolExecutor() as executor:
            # Use lambda to pass additional arguments to uncompress
            executor.map(lambda f: uncompress(f, source, destination), files_to_uncompress)
        return "Done"
    else:
        return "Nothing to uncompress"

In [8]:
def get_onelake_fs(file_system_name):
    class CustomTokenCredential:
        def __init__(self, token):
            self.token = token

        def get_token(self, *scopes, **kwargs):
            return AccessToken(self.token, expires_on=9999999999)  # Set a far future expiration

    credential = CustomTokenCredential(os.getenv('azure_storage_token'))
    service_client = DataLakeServiceClient(account_url=f"https://onelake.dfs.fabric.microsoft.com", credential= credential)
    return service_client.get_file_system_client(file_system_name)

In [9]:
def create_directory_if_not_exist(fs,abfss_directory) :
    directory_client = fs.get_directory_client(abfss_directory)
    try:
        directory_client.get_directory_properties()
        print(f"Directory '{abfss_directory}' exist already.")
    except :
        directory_client.create_directory()
        print(f"Directory '{abfss_directory}' created successfully.")

In [10]:
def List_files_to_upload(fs,local_folder,onelake_folder):
    create_directory_if_not_exist(fs,onelake_folder)
    paths = fs.get_paths(path=onelake_folder)
    onelake_files = [os.path.basename(path.name) for path in paths ]
    local_files=[os.path.basename(x) for x in glob.glob(local_folder+'*.*')]
    filtes_to_upload  = list(  set(local_files) - set(onelake_files))
    filtes_to_upload  = list(dict.fromkeys(filtes_to_upload))
    filtes_to_upload  = sorted(filtes_to_upload, reverse=True)
    filtes_to_upload = [local_folder+'/' + i for i in filtes_to_upload]
    return filtes_to_upload

In [11]:
def upload_file(file_system_client, local_file, remote_directory):

    try:
        # Extract the file name from the local file path
        file_name = os.path.basename(local_file)
        
        # Define the remote file path
        remote_file_path = f"{remote_directory}{file_name}"
        
        # Upload the file
        print(f"Uploading {local_file} to {remote_file_path}")
        file_client = file_system_client.get_file_client(remote_file_path)
        with open(local_file, "rb") as local_file_data:
            file_client.upload_data(local_file_data, overwrite=True)
        print(f"Uploaded {local_file} successfully!")
        return local_file, None  # Return the file and no error
    except Exception as e:
        print(f"Failed to upload {local_file}: {e}")
        return local_file, str(e)  # Return the file and the error message

def upload_files(file_system_client, local_folder, remote_directory, max_workers=5):

    local_files =     List_files_to_upload(file_system_client,local_folder,remote_directory)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit upload tasks to the thread pool
        futures = [
            executor.submit(upload_file, file_system_client, local_file, remote_directory)
            for local_file in local_files
        ]
        
        # Wait for all tasks to complete and handle results
        for future in as_completed(futures):
            local_file, error = future.result()
            if error:
                print(f"Error uploading {local_file}: {error}")
            else:
                print(f"Successfully uploaded {local_file}")

In [12]:
def download_file(file_system_client, remote_file_path, local_directory):
    try:
        # Extract the file name from the remote file path
        file_name = os.path.basename(remote_file_path)
        
        # Define the local file path
        local_file_path = os.path.join(local_directory, file_name)
        
        # Download the file
        print(f"Downloading {remote_file_path} to {local_file_path}")
        file_client = file_system_client.get_file_client(remote_file_path)
        with open(local_file_path, "wb") as local_file_data:
            download_stream = file_client.download_file()
            local_file_data.write(download_stream.readall())
        print(f"Downloaded {remote_file_path} successfully!")
        return remote_file_path, None  # Return the file and no error
    except Exception as e:
        print(f"Failed to download {remote_file_path}: {e}")
        return remote_file_path, str(e)  # Return the file and the error message

def download_remote_storage(file_system_client,Onelake_table, local_directory,Nbr_Files_to_process, max_workers=5):

    files_loaded_in_delta_table = get_loaded_delta_table_files(Onelake_table)
    files_loaded_in_delta_table = [file.replace('.CSV', '.zip') for file in files_loaded_in_delta_table]


    paths = file_system_client.get_paths(path= onelake_file_root_path + local_directory)
    onelake_files = [os.path.basename(path.name) for path in paths ]

    files_to_download = list(set(onelake_files) - set(files_loaded_in_delta_table))
    files_to_download = list(dict.fromkeys(files_to_download))
    remote_files = [onelake_file_root_path + local_directory + file for file in files_to_download][:Nbr_Files_to_process]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit download tasks to the thread pool
        futures = [
            executor.submit(download_file, file_system_client, remote_file,temporary_storage+ local_directory)
            for remote_file in remote_files
        ]
        
        # Wait for all tasks to complete and handle results
        for future in as_completed(futures):
            remote_file, error = future.result()
            if error:
                print(f"Error downloading {remote_file}: {error}")
            else:
                print(f"Successfully downloaded {remote_file}")

In [13]:
def get_loaded_delta_table_files(Table):
 dt =try_get_deltatable(Table,storage_options=None)
 if dt is not None:
    existing_files = duckdb.sql(f""" select distinct file as file from delta_scan('{Table}') """).df()['file'].tolist()
 else:
  existing_files=[]
 return existing_files

In [14]:
def get_Path(Source,Table):
 existing_files= get_loaded_delta_table_files(Table)
 print(len(existing_files))
 list_files_csv=[os.path.basename(x) for x in glob.glob(Source+'*.CSV')]
 list_files_json=[os.path.basename(x) for x in glob.glob(Source+'*.json')]
 filelist = list_files_csv + list_files_json

 files_to_upload = list(set(filelist) - set(existing_files))
 files_to_upload = list(dict.fromkeys(files_to_upload))
 files_to_upload_full_Path = [Source + i for i in files_to_upload]
 return files_to_upload_full_Path[:Nbr_Files_to_process]

**<mark>Transformation Logic</mark>**

**<mark>scada </mark>**

In [15]:
def get_scada(files_to_upload_full_Path):
    raw =duckdb.sql(f"""from read_csv({files_to_upload_full_Path},
    Skip=1,header =0,all_varchar=1,
    columns={{
    'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
    'DUID': 'VARCHAR','INTERVENTION': 'VARCHAR','DISPATCHMODE': 'VARCHAR','AGCSTATUS': 'VARCHAR','INITIALMW': 'VARCHAR',
    'TOTALCLEARED': 'VARCHAR','RAMPDOWNRATE': 'VARCHAR','RAMPUPRATE': 'VARCHAR','LOWER5MIN': 'VARCHAR',
    'LOWER60SEC': 'VARCHAR','LOWER6SEC': 'VARCHAR','RAISE5MIN': 'VARCHAR','RAISE60SEC': 'VARCHAR',
    'RAISE6SEC': 'VARCHAR','MARGINAL5MINVALUE': 'VARCHAR','MARGINAL60SECVALUE': 'VARCHAR',
    'MARGINAL6SECVALUE': 'VARCHAR','MARGINALVALUE': 'VARCHAR','VIOLATION5MINDEGREE': 'VARCHAR',
    'VIOLATION60SECDEGREE': 'VARCHAR','VIOLATION6SECDEGREE': 'VARCHAR','VIOLATIONDEGREE': 'VARCHAR',
    'LOWERREG': 'VARCHAR','RAISEREG': 'VARCHAR','AVAILABILITY': 'VARCHAR','RAISE6SECFLAGS': 'VARCHAR',
    'RAISE60SECFLAGS': 'VARCHAR','RAISE5MINFLAGS': 'VARCHAR','RAISEREGFLAGS': 'VARCHAR',
    'LOWER6SECFLAGS': 'VARCHAR','LOWER60SECFLAGS': 'VARCHAR','LOWER5MINFLAGS': 'VARCHAR',
    'LOWERREGFLAGS': 'VARCHAR','RAISEREGAVAILABILITY': 'VARCHAR','RAISEREGENABLEMENTMAX': 'VARCHAR',
    'RAISEREGENABLEMENTMIN': 'VARCHAR','LOWERREGAVAILABILITY': 'VARCHAR','LOWERREGENABLEMENTMAX': 'VARCHAR',
    'LOWERREGENABLEMENTMIN': 'VARCHAR','RAISE6SECACTUALAVAILABILITY': 'VARCHAR',
    'RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
    'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR',
    'LOWER60SECACTUALAVAILABILITY': 'VARCHAR','LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR'
    }},
    filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
    where I='D' and UNIT ='DUNIT' AND VERSION = 3                  """)

    final_scada=duckdb.sql("""
    select
    UNIT,
    DUID,
    cast(columns(*exclude(DUID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
    parse_filename(filename) as file,
    0 as PRIORITY ,
    cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
    cast(SETTLEMENTDATE as date) as date,
	YEAR (cast (SETTLEMENTDATE as timestamp)) as YEAR
    from raw  """)
    return final_scada.record_batch()


**<mark>Price </mark>**

In [16]:
def get_price(files_to_upload_full_Path):
  raw =duckdb.sql(f"""from read_csv({files_to_upload_full_Path},
  Skip=1,header =0,all_varchar=1,
  columns={{
  'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
  'REGIONID': 'VARCHAR','INTERVENTION': 'VARCHAR','RRP': 'VARCHAR','EEP': 'VARCHAR','ROP': 'VARCHAR','APCFLAG': 'VARCHAR',
  'MARKETSUSPENDEDFLAG': 'VARCHAR','TOTALDEMAND': 'VARCHAR','DEMANDFORECAST': 'VARCHAR','DISPATCHABLEGENERATION': 'VARCHAR',
  'DISPATCHABLELOAD': 'VARCHAR','NETINTERCHANGE': 'VARCHAR','EXCESSGENERATION': 'VARCHAR','LOWER5MINDISPATCH': 'VARCHAR',
  'LOWER5MINIMPORT': 'VARCHAR','LOWER5MINLOCALDISPATCH': 'VARCHAR','LOWER5MINLOCALPRICE': 'VARCHAR','LOWER5MINLOCALREQ': 'VARCHAR',
  'LOWER5MINPRICE': 'VARCHAR','LOWER5MINREQ': 'VARCHAR','LOWER5MINSUPPLYPRICE': 'VARCHAR','LOWER60SECDISPATCH': 'VARCHAR','LOWER60SECIMPORT': 'VARCHAR',
  'LOWER60SECLOCALDISPATCH': 'VARCHAR','LOWER60SECLOCALPRICE': 'VARCHAR','LOWER60SECLOCALREQ': 'VARCHAR','LOWER60SECPRICE': 'VARCHAR',
  'LOWER60SECREQ': 'VARCHAR','LOWER60SECSUPPLYPRICE': 'VARCHAR','LOWER6SECDISPATCH': 'VARCHAR','LOWER6SECIMPORT': 'VARCHAR',
  'LOWER6SECLOCALDISPATCH': 'VARCHAR','LOWER6SECLOCALPRICE': 'VARCHAR','LOWER6SECLOCALREQ': 'VARCHAR','LOWER6SECPRICE': 'VARCHAR',
  'LOWER6SECREQ': 'VARCHAR','LOWER6SECSUPPLYPRICE': 'VARCHAR','RAISE5MINDISPATCH': 'VARCHAR','RAISE5MINIMPORT': 'VARCHAR',
  'RAISE5MINLOCALDISPATCH': 'VARCHAR','RAISE5MINLOCALPRICE': 'VARCHAR','RAISE5MINLOCALREQ': 'VARCHAR','RAISE5MINPRICE': 'VARCHAR',
  'RAISE5MINREQ': 'VARCHAR','RAISE5MINSUPPLYPRICE': 'VARCHAR','RAISE60SECDISPATCH': 'VARCHAR','RAISE60SECIMPORT': 'VARCHAR',
  'RAISE60SECLOCALDISPATCH': 'VARCHAR','RAISE60SECLOCALPRICE': 'VARCHAR','RAISE60SECLOCALREQ': 'VARCHAR','RAISE60SECPRICE': 'VARCHAR',
  'RAISE60SECREQ': 'VARCHAR','RAISE60SECSUPPLYPRICE': 'VARCHAR','RAISE6SECDISPATCH': 'VARCHAR','RAISE6SECIMPORT': 'VARCHAR',
  'RAISE6SECLOCALDISPATCH': 'VARCHAR','RAISE6SECLOCALPRICE': 'VARCHAR','RAISE6SECLOCALREQ': 'VARCHAR','RAISE6SECPRICE': 'VARCHAR',
  'RAISE6SECREQ': 'VARCHAR','RAISE6SECSUPPLYPRICE': 'VARCHAR','AGGREGATEDISPATCHERROR': 'VARCHAR','AVAILABLEGENERATION': 'VARCHAR',
  'AVAILABLELOAD': 'VARCHAR','INITIALSUPPLY': 'VARCHAR','CLEAREDSUPPLY': 'VARCHAR','LOWERREGIMPORT': 'VARCHAR','LOWERREGLOCALDISPATCH': 'VARCHAR',
  'LOWERREGLOCALREQ': 'VARCHAR','LOWERREGREQ': 'VARCHAR','RAISEREGIMPORT': 'VARCHAR','RAISEREGLOCALDISPATCH': 'VARCHAR','RAISEREGLOCALREQ': 'VARCHAR',
  'RAISEREGREQ': 'VARCHAR','RAISE5MINLOCALVIOLATION': 'VARCHAR','RAISEREGLOCALVIOLATION': 'VARCHAR','RAISE60SECLOCALVIOLATION': 'VARCHAR',
  'RAISE6SECLOCALVIOLATION': 'VARCHAR','LOWER5MINLOCALVIOLATION': 'VARCHAR','LOWERREGLOCALVIOLATION': 'VARCHAR','LOWER60SECLOCALVIOLATION': 'VARCHAR',
  'LOWER6SECLOCALVIOLATION': 'VARCHAR','RAISE5MINVIOLATION': 'VARCHAR','RAISEREGVIOLATION': 'VARCHAR','RAISE60SECVIOLATION': 'VARCHAR',
  'RAISE6SECVIOLATION': 'VARCHAR','LOWER5MINVIOLATION': 'VARCHAR','LOWERREGVIOLATION': 'VARCHAR','LOWER60SECVIOLATION': 'VARCHAR',
  'LOWER6SECVIOLATION': 'VARCHAR','RAISE6SECRRP': 'VARCHAR','RAISE6SECROP': 'VARCHAR','RAISE6SECAPCFLAG': 'VARCHAR','RAISE60SECRRP': 'VARCHAR',
  'RAISE60SECROP': 'VARCHAR','RAISE60SECAPCFLAG': 'VARCHAR','RAISE5MINRRP': 'VARCHAR','RAISE5MINROP': 'VARCHAR','RAISE5MINAPCFLAG': 'VARCHAR',
  'RAISEREGRRP': 'VARCHAR','RAISEREGROP': 'VARCHAR','RAISEREGAPCFLAG': 'VARCHAR','LOWER6SECRRP': 'VARCHAR','LOWER6SECROP': 'VARCHAR',
  'LOWER6SECAPCFLAG': 'VARCHAR','LOWER60SECRRP': 'VARCHAR','LOWER60SECROP': 'VARCHAR','LOWER60SECAPCFLAG': 'VARCHAR','LOWER5MINRRP': 'VARCHAR',
  'LOWER5MINROP': 'VARCHAR','LOWER5MINAPCFLAG': 'VARCHAR','LOWERREGRRP': 'VARCHAR','LOWERREGROP': 'VARCHAR','LOWERREGAPCFLAG': 'VARCHAR',
  'RAISE6SECACTUALAVAILABILITY': 'VARCHAR','RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
  'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR','LOWER60SECACTUALAVAILABILITY': 'VARCHAR',
  'LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR','LORSURPLUS': 'VARCHAR','LRCSURPLUS': 'VARCHAR',
  }},
  filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
  where I='D' and UNIT ='DREGION' AND VERSION = 3
                  """)
  final_price=duckdb.sql("""
    select
    UNIT,
    REGIONID,
    cast(columns(*exclude(REGIONID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
    parse_filename(filename) as file,
    0 as PRIORITY ,
    cast (SETTLEMENTDATE as TIMESTAMPTZ) as SETTLEMENTDATE,
    cast(SETTLEMENTDATE as date) as date,
	  YEAR (cast (SETTLEMENTDATE as timestamp)) as YEAR
    from raw  """)
  return final_price.record_batch()

**<mark>WA </mark>**

In [17]:
def get_wa(files_to_upload_full_Path):
    final_wa=duckdb.sql(f"""
        select
        data.code as DUID,
        data.quantity as INITIALMW,
        cast(data.dispatchInterval AS TIMESTAMPTZ) SETTLEMENTDATE  ,
        cast(SETTLEMENTDATE as date) as date,
        parse_filename(filename) as file,
        0 as PRIORITY ,
        YEAR (cast (data.dispatchInterval as timestamp)) as YEAR
        from (select unnest(data.facilityScadaDispatchIntervals) as data,filename
        from read_json_auto({files_to_upload_full_Path},filename=1)) """)
    df_wa=final_wa.record_batch()
    return df_wa

# Process Data

In [18]:
fs = get_onelake_fs(f"{ws}")

In [19]:
%%time
Web_Path                = "https://nemweb.com.au/Reports/Current/Daily_Reports/"
Zip_Path                = "/0_Source/Daily_Reports/"
uncompressed_Path       = "/1_Transform/CSV/Daily_Reports/"
Onelake_table           = table_base_url+'price'
#############################################
download_web(Web_Path,Zip_Path,Nbr_Files_to_process)
upload_files(fs, temporary_storage+Zip_Path, onelake_file_root_path + Zip_Path , max_workers= cpu_count())
download_remote_storage(fs,Onelake_table ,  Zip_Path,Nbr_Files_to_process, max_workers= cpu_count())
#############################################
unzip(temporary_storage+Zip_Path,temporary_storage+uncompressed_Path)
files_to_upload_full_Path = get_Path(temporary_storage+uncompressed_Path,Onelake_table)
if len(files_to_upload_full_Path) >0 :
  df = get_price(files_to_upload_full_Path)
  write_deltalake(Onelake_table, df,mode="append",engine=engine)
else:
  print('all loaded already')

Directory 'power.Lakehouse/Files/aemo/0_Source/Daily_Reports/' exist already.
0 New File Downloaded
Directory 'power.Lakehouse/Files/aemo/0_Source/Daily_Reports/' exist already.
0 new file(s) to uncompress.
2471
all loaded already
CPU times: user 2.1 s, sys: 197 ms, total: 2.29 s
Wall time: 12.9 s


In [20]:
%%time
Web_Path                = "https://nemweb.com.au/Reports/Current/Daily_Reports/"
Zip_Path                = "/0_Source/Daily_Reports/"
uncompressed_Path       = "/1_Transform/CSV/Daily_Reports/"
Onelake_table           = table_base_url+'scada'
#############################################
download_web(Web_Path,Zip_Path,Nbr_Files_to_process)
upload_files(fs, temporary_storage+Zip_Path, onelake_file_root_path + Zip_Path , max_workers= cpu_count())
download_remote_storage(fs,Onelake_table ,  Zip_Path,Nbr_Files_to_process, max_workers= cpu_count())
#############################################
unzip(temporary_storage+Zip_Path,temporary_storage+uncompressed_Path)
files_to_upload_full_Path = get_Path(temporary_storage+uncompressed_Path,Onelake_table)
if len(files_to_upload_full_Path) >0 :
    df = get_scada(files_to_upload_full_Path)
    write_deltalake(Onelake_table, df,mode="append",partition_by=['YEAR'],engine= engine)
    del df
else:
  print('all loaded already')

Directory 'power.Lakehouse/Files/aemo/0_Source/Daily_Reports/' exist already.
0 New File Downloaded
Directory 'power.Lakehouse/Files/aemo/0_Source/Daily_Reports/' exist already.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

0 new file(s) to uncompress.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2471
all loaded already
CPU times: user 22.7 s, sys: 453 ms, total: 23.1 s
Wall time: 1min 40s


In [21]:
%%time
Web_Path                = "https://data.wa.aemo.com.au/public/market-data/wemde/facilityScada/previous/"
Zip_Path                = "/0_Source/wa/"
uncompressed_Path       = "/1_Transform/JSON/WA/facilityScada/"
Onelake_table           = table_base_url+'wa'
#############################################
download_web(Web_Path,Zip_Path,Nbr_Files_to_process)
upload_files(fs, temporary_storage+Zip_Path, onelake_file_root_path + Zip_Path ,  max_workers= cpu_count())
download_remote_storage(fs,Onelake_table ,  Zip_Path, Nbr_Files_to_process , max_workers= cpu_count())
#############################################
unzip(temporary_storage+Zip_Path,temporary_storage+uncompressed_Path)
files_to_upload_full_Path = get_Path(temporary_storage+uncompressed_Path,Onelake_table)
if len(files_to_upload_full_Path) >0 :
  df = get_wa(files_to_upload_full_Path)
  write_deltalake(Onelake_table, df,mode="append",partition_by=['YEAR'],engine= engine)
  del df
else:
  print('all loaded already')

Directory 'power.Lakehouse/Files/aemo/0_Source/wa/' exist already.
0 New File Downloaded
Directory 'power.Lakehouse/Files/aemo/0_Source/wa/' exist already.
Downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20241108.zip successfully!
Successfully downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20241108.zip
Downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20231023.zip successfully!
Successfully downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20231023.zip
Downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20241215.zip successfully!
Successfully downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20241215.zip
Downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20240725.zip successfully!
Successfully downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20240725.zip
Downloaded power.Lakehouse/Files/aemo/0_Source/wa/FacilityScada_20240313.zip successfully!
Successfully downloaded power